In [9]:
pow(5,-5+10,11)

1

In [38]:
class FinitFieldElement():
    def __init__(self, number:int, p:int):
        FinitFieldElement.check_errors(number, p)
        self.value = number
        self.p = p
        
    def __add__(self, y):
        self.check_field(y)
        return FinitFieldElement((self.value + y.value)%self.p, self.p)
    
    def __sub__(self, y):
        self.check_field(y)
        return FinitFieldElement((self.value - y.value)%self.p, self.p)
    
    def __mul__(self, y):
        self.check_field(y)
        return FinitFieldElement((self.value * y.value)%self.p, self.p)

    def __truediv__(self, y, inv_mul_method='euclidean'):
        if inv_mul_method != 'euclidean' and inv_mul_method != 'fermat':
            raise ValueError("The method to compute the multiplicative inverse must be 'euclidean' or 'fermat'")
        self.check_field(y)
            
        inv_mul_y = y.extended_euclidean_algorithm() if inv_mul_method =='euclidean' else y.fermats_little_theorem()
        return FinitFieldElement((self.value * inv_mul_y)%self.p,self.p)
    
    def __pow__(self, exponent):
        exp_aux = exponent
        while exp_aux < 0:
            exp_aux += self.p - 1
        return FinitFieldElement(pow(self.value,exp_aux,self.p), self.p)
    
    def __eq__(self, y):
        return (self.value == y.value) and (self.p == y.p)
    
    def check_errors(number,p):
        if not isinstance(number, int):
            raise TypeError("The number must be integer")
        if not(0 <= number < p):
            raise ValueError("The number must be greater than or equal to 0 and lower than the prime number P")
            
    def check_field(self, y):
        if y.p != self.p:
            raise TypeError("The numbers must belong to the same finit field")
            
    def extended_euclidean_algorithm(self):
        r0, r1 = self.value, self.p
        s0, s1 = 1, 0
        t0, t1 = 0, 1
    
        while r1 != 0:
            q = r0 // r1
            r0, r1 = r1, r0 % r1
            s0, s1 = s1, s0 - q * s1
            t0, t1 = t1, t0 - q * t1
        if r0 != 1:
            raise ValueError(f"The value {self.value} has not inverse of multiplication in {self.p}-finit field")
    
        return s0 % self.p
    
    def fermats_little_theorem(self):
        return (self.value ** (self.p - 2))%self.p

In [40]:
x = FinitFieldElement(5,11)
y = FinitFieldElement(8,11)
z = FinitFieldElement(5,11)


x==z

True

In [116]:
class EllipticCurveElement():
    def __init__(self,x,y,a,b):
        if not(x==None and y==None):
            EllipticCurveElement.check_point(x,y,a,b)
        self.x = x
        self.y = y
        self.a = a
        self.b = b
        
    def __add__(self, other_point):
        self.check_same_curve(other_point)
        
        if self.is_inf(): return other_point
        elif other_point.is_inf(): return self
        elif self.x==other_point.x and self.y!=other_point.y: return EllipticCurveElement(None,None,self.a,self.b)
    
        s = (other_point.y - self.y)/(other_point.x - self.x) if self!=other_point else (3*(self.x**2)+self.a)/(2*self.y)
        x = s**2 - self.x - other_point.x
        y = s * (self.x - x) - self.y
        return EllipticCurveElement(x,y,self.a,self.b)
        
        
    def __eq__(self, other_point):
        return (self.x == other_point.x) and (self.y == other_point.y) 
    
    
    def __ne__(self, other_point):
        return not (self==other_point)
        
    def is_inf(self):
        return (self.x==None) and (self.y==None)
            
    def check_same_curve(self, other_point):
        if self.a != other_point.a or self.b != other_point.b:
            raise ValueError(f"(The given points are not in the same elliptic curve")
            
    def check_point(x,y,a,b):
        if (x==None) ^ (y==None):
            raise ValueError(f"Invalid point")
        # if abs((y**2) - (x**3 + a * x + b)) > 0.000001:
        #     raise ValueError(f"The point ({x},{y}) is not on the elliptic curve")            
        if y**2 != x**3 + a * x + b:
            raise ValueError(f"The point ({x},{y}) is not on the elliptic curve")
        

In [115]:
p1 = EllipticCurveElement(0,2.64575131106459059,0,7)
p2 = EllipticCurveElement(0,-2.64575131106459059,0,7)

print((p1+p2).y)

None
